In [2]:
import pandas as pd
import seaborn as sns
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

In [3]:
df = sns.load_dataset('titanic')
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


In [7]:
# check NaN count 
df.isna().sum()

survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64

In [8]:
# delete deck and embark_town columns 
rdf = df.drop(["deck", "embark_town"], axis = 1)

In [9]:
rdf.columns.values

array(['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'embarked', 'class', 'who', 'adult_male', 'alive', 'alone'],
      dtype=object)

In [11]:
# delete rows with age as NaN
rdf = rdf.dropna(subset=["age"], axis = 0)

In [12]:
len(rdf)

714

In [16]:
most_freq = rdf["embarked"].value_counts().idxmax()
most_freq

'S'

In [17]:
# switch NaN in embarked to S 
rdf["embarked"].fillna(most_freq, inplace=True)

In [19]:
rdf.isna().sum()

survived      0
pclass        0
sex           0
age           0
sibsp         0
parch         0
fare          0
embarked      0
class         0
who           0
adult_male    0
alive         0
alone         0
dtype: int64

In [40]:
ndf = rdf[["survived", "pclass", "sex", "age", "sibsp", "parch", "embarked"]]

In [41]:
ndf.head()

,survived,pclass,sex,age,sibsp,parch,embarked
0,0,3,male,22.0,1,0,S
1,1,1,female,38.0,1,0,C
2,1,3,female,26.0,0,0,S
3,1,1,female,35.0,1,0,S
4,0,3,male,35.0,0,0,S


Need to show sex and embarked columns in a numerical way 

One-hot Encoding
- the categorical parameters will prepare separate columns for both Male and Female labels. So, wherever there is a Male, the value will be 1 in the Male column and 0 in the Female column, and vice-versa. 

In [42]:
onehot_sex = pd.get_dummies(ndf["sex"], dtype="int")

In [44]:
onehot_sex

,female,male
0,0,1
1,1,0
2,1,0
3,1,0
4,0,1
...,...,...
885,1,0
886,0,1
887,1,0
889,0,1


In [45]:
ndf = pd.concat([ndf, onehot_sex], axis=1)

In [46]:
ndf.head()

,survived,pclass,sex,age,sibsp,parch,embarked,female,male
0,0,3,male,22.0,1,0,S,0,1
1,1,1,female,38.0,1,0,C,1,0
2,1,3,female,26.0,0,0,S,1,0
3,1,1,female,35.0,1,0,S,1,0
4,0,3,male,35.0,0,0,S,0,1


In [47]:
onehot_embarked = pd.get_dummies(ndf["embarked"], prefix="town", dtype="int")

In [48]:
onehot_embarked.head()

,town_C,town_Q,town_S
0,0,0,1
1,1,0,0
2,0,0,1
3,0,0,1
4,0,0,1


In [49]:
ndf = pd.concat([ndf, onehot_embarked], axis=1)

In [50]:
ndf.head()

,survived,pclass,sex,age,sibsp,parch,embarked,female,male,town_C,town_Q,town_S
0,0,3,male,22.0,1,0,S,0,1,0,0,1
1,1,1,female,38.0,1,0,C,1,0,1,0,0
2,1,3,female,26.0,0,0,S,1,0,0,0,1
3,1,1,female,35.0,1,0,S,1,0,0,0,1
4,0,3,male,35.0,0,0,S,0,1,0,0,1


In [51]:
ndf = ndf.drop(["sex", "embarked"], axis = 1)

In [52]:
ndf.head()

,survived,pclass,age,sibsp,parch,female,male,town_C,town_Q,town_S
0,0,3,22.0,1,0,0,1,0,0,1
1,1,1,38.0,1,0,1,0,1,0,0
2,1,3,26.0,0,0,1,0,0,0,1
3,1,1,35.0,1,0,1,0,0,0,1
4,0,3,35.0,0,0,0,1,0,0,1


Split Data

In [54]:
x = ndf.drop("survived" , axis=1)
y = ndf["survived"]

In [56]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, stratify=y, random_state=12)

In [57]:
len(x_train), len(x_test)

(499, 215)

KNN Classification 

In [58]:
knn = KNeighborsClassifier(n_neighbors=5)

In [59]:
knn.fit(x_train, y_train)

KNeighborsClassifier()

In [60]:
y_pred = knn.predict(x_test)

In [61]:
pd.DataFrame({"ans":y_test, "pred":y_pred})

,ans,pred
441,0,0
189,0,0
394,1,1
56,1,1
645,1,1
...,...,...
442,0,0
796,1,1
333,0,0
227,0,0


In [63]:
from sklearn.metrics import confusion_matrix


knn_matrix = confusion_matrix(y_test, y_pred)

In [64]:
print(knn_matrix)

[[104  24]
 [ 23  64]]
